# మిస్ట్రాల్ మోడల్స్‌తో నిర్మాణం

## పరిచయం

ఈ పాఠం కవర్ చేస్తుంది:  
- వివిధ మిస్ట్రాల్ మోడల్స్‌ను అన్వేషించడం  
- ప్రతి మోడల్ కోసం ఉపయోగాల మరియు పరిస్థితులను అర్థం చేసుకోవడం  
- ప్రతి మోడల్ యొక్క ప్రత్యేక లక్షణాలను చూపించే కోడ్ నమూనాలు.


## మిస్ట్రాల్ మోడల్స్

ఈ పాఠంలో, మేము 3 వేర్వేరు మిస్ట్రాల్ మోడల్స్‌ను అన్వేషించబోతున్నాము:  
**మిస్ట్రాల్ లార్జ్**, **మిస్ట్రాల్ స్మాల్** మరియు **మిస్ట్రాల్ నేమో**.

ఈ మోడల్స్ ప్రతి ఒక్కటి గిట్హబ్ మోడల్ మార్కెట్‌ప్లేస్‌లో ఉచితంగా అందుబాటులో ఉన్నాయి. ఈ నోట్‌బుక్‌లోని కోడ్ ఈ మోడల్స్‌ను ఉపయోగించి కోడ్‌ను నడిపిస్తుంది. గిట్హబ్ మోడల్స్‌ను ఉపయోగించి [AI మోడల్స్‌తో ప్రోటోటైపింగ్](https://docs.github.com/en/github-models/prototyping-with-ai-models?WT.mc_id=academic-105485-koreyst) గురించి మరిన్ని వివరాలు ఇక్కడ ఉన్నాయి.


## Mistral Large 2 (2407)
Mistral Large 2 ప్రస్తుతం Mistral నుండి ప్రధాన మోడల్ మరియు ఇది ఎంటర్ప్రైజ్ ఉపయోగానికి రూపొందించబడింది.

ఈ మోడల్ అసలు Mistral Large కి ఒక అప్‌గ్రేడ్, ఇది అందిస్తుంది
-  పెద్ద కాంటెక్స్ట్ విండో - 128k వర్సెస్ 32k
-  గణితం మరియు కోడింగ్ పనులపై మెరుగైన పనితీరు - 76.9% సగటు ఖచ్చితత్వం వర్సెస్ 60.4%
-  బహుభాషా పనితీరు పెరిగింది - భాషలు: ఆంగ్లం, ఫ్రెంచ్, జర్మన్, స్పానిష్, ఇటాలియన్, పోర్చుగీస్, డచ్, రష్యన్, చైనీస్, జపనీస్, కొరియన్, అరబిక్, మరియు హిందీ.

ఈ లక్షణాలతో, Mistral Large ఈ క్రింది వాటిలో మెరుగ్గా పనిచేస్తుంది
- *Retrieval Augmented Generation (RAG)* - పెద్ద కాంటెక్స్ట్ విండో కారణంగా
- *Function Calling* - ఈ మోడల్ స్థానిక ఫంక్షన్ కాలింగ్ కలిగి ఉంది, ఇది బాహ్య టూల్స్ మరియు APIs తో ఇంటిగ్రేషన్ అనుమతిస్తుంది. ఈ కాల్స్ సమాంతరంగా లేదా వరుసగా ఒకదాని తర్వాత ఒకటి చేయవచ్చు.
- *Code Generation* - ఈ మోడల్ Python, Java, TypeScript మరియు C++ జనరేషన్ లో మెరుగ్గా పనిచేస్తుంది.


### మిస్ట్రాల్ లార్జ్ 2 ఉపయోగించి RAG ఉదాహరణ


ఈ ఉదాహరణలో, మేము Mistral Large 2 ను ఉపయోగించి ఒక టెక్స్ట్ డాక్యుమెంట్ పై RAG నమూనాను నడుపుతున్నాము. ప్రశ్న కొరియన్ భాషలో రాయబడింది మరియు రచయిత కళాశాలకి ముందు చేసిన కార్యకలాపాల గురించి అడుగుతుంది.

ఇది Cohere Embeddings Model ను ఉపయోగించి టెక్స్ట్ డాక్యుమెంట్ మరియు ప్రశ్న యొక్క ఎంబెడ్డింగ్స్ సృష్టిస్తుంది. ఈ నమూనాకు, faiss Python ప్యాకేజీని వెక్టర్ స్టోర్ గా ఉపయోగిస్తుంది.

Mistral మోడల్ కు పంపబడిన ప్రాంప్ట్ లో ప్రశ్నలు మరియు ప్రశ్నకు సమానమైన తిరిగి పొందిన భాగాలు రెండూ ఉంటాయి. ఆ తర్వాత మోడల్ సహజ భాషలో సమాధానం అందిస్తుంది.


In [50]:
pip install faiss-cpu

Note: you may need to restart the kernel to use updated packages.


In [51]:
import requests
import numpy as np
import faiss
import os

from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import SystemMessage, UserMessage
from azure.core.credentials import AzureKeyCredential
from azure.ai.inference import EmbeddingsClient

endpoint = "https://models.inference.ai.azure.com"
model_name = "Mistral-large"
token = os.environ["GITHUB_TOKEN"]

client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(token),
)

response = requests.get('https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/paul_graham/paul_graham_essay.txt')
text = response.text

chunk_size = 2048
chunks = [text[i:i + chunk_size] for i in range(0, len(text), chunk_size)]
len(chunks)

embed_model_name = "cohere-embed-v3-multilingual" 

embed_client = EmbeddingsClient(
        endpoint=endpoint,
        credential=AzureKeyCredential(token)
)

embed_response = embed_client.embed(
    input=chunks,
    model=embed_model_name
)



text_embeddings = []
for item in embed_response.data:
    length = len(item.embedding)
    text_embeddings.append(item.embedding)
text_embeddings = np.array(text_embeddings)


d = text_embeddings.shape[1]
index = faiss.IndexFlatL2(d)
index.add(text_embeddings)

question = "저자가 대학에 오기 전에 주로 했던 두 가지 일은 무엇이었나요?？"

question_embedding = embed_client.embed(
    input=[question],
    model=embed_model_name
)

question_embeddings = np.array(question_embedding.data[0].embedding)


D, I = index.search(question_embeddings.reshape(1, -1), k=2) # distance, index
retrieved_chunks = [chunks[i] for i in I.tolist()[0]]

prompt = f"""
Context information is below.
---------------------
{retrieved_chunks}
---------------------
Given the context information and not prior knowledge, answer the query.
Query: {question}
Answer:
"""


chat_response = client.complete(
    messages=[
        SystemMessage(content="You are a helpful assistant."),
        UserMessage(content=prompt),
    ],
    temperature=1.0,
    top_p=1.0,
    max_tokens=1000,
    model=model_name
)

print(chat_response.choices[0].message.content)

The author primarily engaged in two activities before college: writing and programming. In terms of writing, they wrote short stories, albeit not very good ones, with minimal plot and characters expressing strong feelings. For programming, they started writing programs on the IBM 1401 used for data processing during their 9th grade, at the age of 13 or 14. They used an early version of Fortran and typed programs on punch cards, later loading them into the card reader to run the program.


## మిస్ట్రాల్ స్మాల్  
మిస్ట్రాల్ స్మాల్ అనేది ప్రీమియర్/ఎంటర్ప్రైజ్ వర్గంలో మిస్ట్రాల్ కుటుంబంలోని మరో మోడల్. పేరుని బట్టి అర్థం చేసుకోవచ్చు, ఈ మోడల్ ఒక చిన్న భాషా మోడల్ (SLM). మిస్ట్రాల్ స్మాల్ ఉపయోగించడంలో లాభాలు:  
- మిస్ట్రాల్ LLMs లాంటి మిస్ట్రాల్ లార్జ్ మరియు NeMo తో పోల్చితే ఖర్చు ఆదా - 80% ధర తగ్గింపు  
- తక్కువ లేటెన్సీ - మిస్ట్రాల్ LLMs తో పోల్చితే వేగవంతమైన స్పందన  
- సౌలభ్యం - అవసరమైన వనరులపై తక్కువ పరిమితులతో వివిధ వాతావరణాల్లో అమలు చేయవచ్చు.  

మిస్ట్రాల్ స్మాల్ ఈ క్రింది పనులకు అద్భుతం:  
- సారాంశం, భావ విశ్లేషణ మరియు అనువాదం వంటి టెక్స్ట్ ఆధారిత పనులు  
- ఖర్చు సమర్థత కారణంగా తరచుగా అభ్యర్థనలు చేసే అప్లికేషన్లు  
- సమీక్ష మరియు కోడ్ సూచనల వంటి తక్కువ లేటెన్సీ కోడ్ పనులు


## Mistral Small మరియు Mistral Large ను పోల్చడం

Mistral Small మరియు Large మధ్య ఆలస్యం తేడాలను చూపించడానికి, క్రింది సెల్స్‌ను నడపండి.

3-5 సెకన్ల మధ్య ప్రతిస్పందన సమయాలలో తేడా కనిపించాలి. అదే ప్రాంప్ట్‌పై ప్రతిస్పందన పొడవులు మరియు శైలి కూడా గమనించండి.


In [ ]:
import os 
endpoint = "https://models.inference.ai.azure.com"
model_name = "Mistral-small"
token = os.environ["GITHUB_TOKEN"]

client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(token),
)

response = client.complete(
    messages=[
        SystemMessage(content="You are a helpful coding assistant."),
        UserMessage(content="Can you write a Python function to the fizz buzz test?"),
    ],
    temperature=1.0,
    top_p=1.0,
    max_tokens=1000,
    model=model_name
)

print(response.choices[0].message.content)

In [ ]:
import os
from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import SystemMessage, UserMessage
from azure.core.credentials import AzureKeyCredential

endpoint = "https://models.inference.ai.azure.com"
model_name = "Mistral-large"
token = os.environ["GITHUB_TOKEN"]

client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(token),
)

response = client.complete(
    messages=[
        SystemMessage(content="You are a helpful coding assistant."),
        UserMessage(content="Can you write a Python function to the fizz buzz test?"),
    ],
    temperature=1.0,
    top_p=1.0,
    max_tokens=1000,
    model=model_name
)

print(response.choices[0].message.content)

## Mistral NeMo

ఈ పాఠంలో చర్చించిన మిగతా రెండు మోడళ్లతో పోల్చితే, Mistral NeMo మాత్రమే Apache2 లైసెన్స్‌తో ఉచిత మోడల్.

ఇది Mistral నుండి వచ్చిన పూర్వపు ఓపెన్ సోర్స్ LLM అయిన Mistral 7Bకి అప్‌గ్రేడ్‌గా భావించబడుతుంది.

NeMo మోడల్ యొక్క కొన్ని ఇతర లక్షణాలు:

- *మరింత సమర్థవంతమైన టోకనైజేషన్:* ఈ మోడల్ సాధారణంగా ఉపయోగించే tiktoken కంటే Tekken టోకనైజర్‌ను ఉపయోగిస్తుంది. ఇది మరిన్ని భాషలు మరియు కోడ్‌పై మెరుగైన పనితీరును అందిస్తుంది.

- *ఫైన్‌ట్యూనింగ్:* బేస్ మోడల్ ఫైన్‌ట్యూనింగ్ కోసం అందుబాటులో ఉంది. ఇది ఫైన్‌ట్యూనింగ్ అవసరం ఉన్న వినియోగ సందర్భాలకు మరింత అనుకూలతను ఇస్తుంది.

- *నేటివ్ ఫంక్షన్ కాలింగ్* - Mistral Large లాగా, ఈ మోడల్ ఫంక్షన్ కాలింగ్‌పై శిక్షణ పొందింది. ఇది మొదటి ఓపెన్ సోర్స్ మోడళ్లలో ఒకటిగా ప్రత్యేకతను కలిగి ఉంది.


## Mistral NeMo

ఈ పాఠంలో చర్చించిన మిగతా రెండు మోడళ్లతో పోల్చితే, Mistral NeMo మాత్రమే Apache2 లైసెన్స్‌తో ఉచిత మోడల్.

ఇది Mistral నుండి వచ్చిన పూర్వపు ఓపెన్ సోర్స్ LLM అయిన Mistral 7Bకి అప్‌గ్రేడ్‌గా భావించబడుతుంది.

NeMo మోడల్ యొక్క కొన్ని ఇతర లక్షణాలు:

- *మరింత సమర్థవంతమైన టోకనైజేషన్:* ఈ మోడల్ సాధారణంగా ఉపయోగించే tiktoken కంటే Tekken టోకనైజర్‌ను ఉపయోగిస్తుంది. ఇది మరిన్ని భాషలు మరియు కోడ్‌లపై మెరుగైన పనితీరును అందిస్తుంది.

- *ఫైన్‌ట్యూనింగ్:* బేస్ మోడల్ ఫైన్‌ట్యూనింగ్ కోసం అందుబాటులో ఉంది. ఇది ఫైన్‌ట్యూనింగ్ అవసరం ఉన్న వినియోగ సందర్భాలకు మరింత సౌలభ్యాన్ని ఇస్తుంది.

- *నేటివ్ ఫంక్షన్ కాలింగ్* - Mistral Large లాగా, ఈ మోడల్ కూడా ఫంక్షన్ కాలింగ్‌పై శిక్షణ పొందింది. ఇది మొదటి ఓపెన్ సోర్స్ మోడళ్లలో ఒకటిగా ప్రత్యేకతను కలిగి ఉంది.


### టోకనైజర్లను పోల్చడం

ఈ నమూనాలో, మిస్ట్రాల్ నేమో టోకనైజేషన్‌ను మిస్ట్రాల్ లార్జ్‌తో పోల్చి ఎలా నిర్వహిస్తుందో చూద్దాం.

రెండు నమూనాలు కూడా అదే ప్రాంప్ట్ తీసుకుంటాయి కానీ నేమో మిస్ట్రాల్ లార్జ్ కంటే తక్కువ టోకెన్లను తిరిగి ఇస్తుందని మీరు చూడగలరు.


In [11]:
pip install mistral-common

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 63.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 797.0/797.0 kB 15.1 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [12]:
# Import needed packages:
from mistral_common.protocol.instruct.messages import (
    UserMessage,
)
from mistral_common.protocol.instruct.request import ChatCompletionRequest
from mistral_common.protocol.instruct.tool_calls import (
    Function,
    Tool,
)
from mistral_common.tokens.tokenizers.mistral import MistralTokenizer

# Load Mistral tokenizer

model_name = "open-mistral-nemo	"

tokenizer = MistralTokenizer.from_model(model_name)

# Tokenize a list of messages
tokenized = tokenizer.encode_chat_completion(
    ChatCompletionRequest(
        tools=[
            Tool(
                function=Function(
                    name="get_current_weather",
                    description="Get the current weather",
                    parameters={
                        "type": "object",
                        "properties": {
                            "location": {
                                "type": "string",
                                "description": "The city and state, e.g. San Francisco, CA",
                            },
                            "format": {
                                "type": "string",
                                "enum": ["celsius", "fahrenheit"],
                                "description": "The temperature unit to use. Infer this from the users location.",
                            },
                        },
                        "required": ["location", "format"],
                    },
                )
            )
        ],
        messages=[
            UserMessage(content="What's the weather like today in Paris"),
        ],
        model=model_name,
    )
)
tokens, text = tokenized.tokens, tokenized.text

# Count the number of tokens
print(len(tokens))

128


In [13]:
# Import needed packages:
from mistral_common.protocol.instruct.messages import (
    UserMessage,
)
from mistral_common.protocol.instruct.request import ChatCompletionRequest
from mistral_common.protocol.instruct.tool_calls import (
    Function,
    Tool,
)
from mistral_common.tokens.tokenizers.mistral import MistralTokenizer

# Load Mistral tokenizer

model_name = "mistral-large-latest"

tokenizer = MistralTokenizer.from_model(model_name)

# Tokenize a list of messages
tokenized = tokenizer.encode_chat_completion(
    ChatCompletionRequest(
        tools=[
            Tool(
                function=Function(
                    name="get_current_weather",
                    description="Get the current weather",
                    parameters={
                        "type": "object",
                        "properties": {
                            "location": {
                                "type": "string",
                                "description": "The city and state, e.g. San Francisco, CA",
                            },
                            "format": {
                                "type": "string",
                                "enum": ["celsius", "fahrenheit"],
                                "description": "The temperature unit to use. Infer this from the users location.",
                            },
                        },
                        "required": ["location", "format"],
                    },
                )
            )
        ],
        messages=[
            UserMessage(content="What's the weather like today in Paris"),
        ],
        model=model_name,
    )
)
tokens, text = tokenized.tokens, tokenized.text

# Count the number of tokens
print(len(tokens))

135


## ఇక్కడే నేర్చుకోవడం ఆగదు, ప్రయాణాన్ని కొనసాగించండి

ఈ పాఠం పూర్తి చేసిన తర్వాత, మీ జనరేటివ్ AI జ్ఞానాన్ని మరింత పెంచుకోవడానికి మా [Generative AI Learning collection](https://aka.ms/genai-collection?WT.mc_id=academic-105485-koreyst) ను చూడండి!


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**అస్పష్టత**:  
ఈ పత్రాన్ని AI అనువాద సేవ [Co-op Translator](https://github.com/Azure/co-op-translator) ఉపయోగించి అనువదించబడింది. మేము ఖచ్చితత్వానికి ప్రయత్నించినప్పటికీ, ఆటోమేటెడ్ అనువాదాల్లో పొరపాట్లు లేదా తప్పిదాలు ఉండవచ్చు. అసలు పత్రం దాని స్వదేశీ భాషలోనే అధికారిక మూలంగా పరిగణించాలి. ముఖ్యమైన సమాచారానికి, ప్రొఫెషనల్ మానవ అనువాదం సిఫార్సు చేయబడుతుంది. ఈ అనువాదం వాడకంలో ఏర్పడిన ఏవైనా అపార్థాలు లేదా తప్పుదారుల కోసం మేము బాధ్యత వహించము.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
